In [1]:
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt
import os
import h5py
import gc
import math

# third-party toolkits for the first implementation to see what the results look like, 
# will implement the K-Means by my self following the first implementation
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from hmmlearn.hmm import GaussianHMM, GMMHMM

In [2]:
num_state = 8
num_mix = 3

In [3]:
num_features_after_pca = 60

In [7]:

#read data
data_set_folder = 'Data_set_with_EEG_step_1.0_3PCA_3x20_intact/'

TN_subjectNum = 22
subject_num_list_for_test = [21, 22] #change every time


# construct the training set
are_training_sets_not_initialized = True
for subject_num in range(1, TN_subjectNum + 1):
    print('Processing subject #', subject_num)
    if subject_num in subject_num_list_for_test:
            continue
    subject_num_str = str(subject_num + 100000)[-2:]
    y_path = data_set_folder + 'sj_' + subject_num_str + '_Y.mat'
    y_list_mat = h5py.File(y_path, mode='r')
    y_list = np.transpose(y_list_mat['y_list'][:].astype(np.float32))

    x_path = data_set_folder + 'sj_' + subject_num_str + '_X.mat'
    x_matrix_mat = h5py.File(x_path, mode='r')
    x_matrix = np.transpose(x_matrix_mat['x_matrix'][:].astype(np.float32))
    num_features, num_samples_of_each_subject = x_matrix.shape


    # balance the samples of each subject
    count0 = 0
    count1 = 0
    for i in range(len(y_list)):
        if y_list[i][0] == 0:
            count0 = count0 + 1
        else:
            count1 = count1 + 1

    x_matrix_y0 = np.zeros((num_features, count0))
    x_matrix_y1 = np.zeros((num_features, count1))
    x_matrix_y0_pointer = 0
    x_matrix_y1_pointer = 0

    for column_num in range(num_samples_of_each_subject):
        x_sample = x_matrix[:, column_num]
        if y_list[column_num][0] == 0:
            x_matrix_y0[:, x_matrix_y0_pointer] = x_sample
            x_matrix_y0_pointer = x_matrix_y0_pointer + 1
        else:
            x_matrix_y1[:, x_matrix_y1_pointer] = x_sample
            x_matrix_y1_pointer = x_matrix_y1_pointer + 1

    # shuffle all columns of x_matrix_y0
    perm = np.arange(count0)
    np.random.shuffle(perm)
    x_matrix_y0 = x_matrix_y0.T[perm]
    x_matrix_y0 = x_matrix_y0.T

    # shuffle all columns of x_matrix_y1
    perm = np.arange(count1)
    np.random.shuffle(perm)
    x_matrix_y1 = x_matrix_y1.T[perm]
    x_matrix_y1 = x_matrix_y1.T

    if count0 < count1:
        x_matrix_y1 = x_matrix_y1[:, :count0]
    else:
        x_matrix_y0 = x_matrix_y0[:, :count1]

    if are_training_sets_not_initialized:
        training_set_y0 = x_matrix_y0
        training_set_y1 = x_matrix_y1
        are_training_sets_not_initialized = False
    else:
        training_set_y0 = np.hstack((training_set_y0, x_matrix_y0))
        training_set_y1 = np.hstack((training_set_y1, x_matrix_y1))


Processing subject # 1
Processing subject # 2
Processing subject # 3
Processing subject # 4
Processing subject # 5
Processing subject # 6
Processing subject # 7
Processing subject # 8
Processing subject # 9
Processing subject # 10
Processing subject # 11
Processing subject # 12
Processing subject # 13
Processing subject # 14
Processing subject # 15
Processing subject # 16
Processing subject # 17
Processing subject # 18
Processing subject # 19
Processing subject # 20
Processing subject # 21
Processing subject # 22


In [8]:
training_set_y0.shape

(43080, 5491)

In [9]:
#run with 4000 samples from y0 and y1
# training_set_y0 = training_set_y0[:, 0:2000] 
# training_set_y1 = training_set_y1[:, 0:2000]

In [10]:
y0_feature_list = []
for sample_n in range(len(training_set_y0[0])):
    features = np.reshape(training_set_y0[:, sample_n], (num_features_after_pca, -1), 'F')
    y0_feature_list.append(features)
del training_set_y0
gc.collect()

y1_feature_list = []
for sample_n in range(len(training_set_y1[0])):
    features = np.reshape(training_set_y1[:, sample_n], (num_features_after_pca, -1), 'F')
    y1_feature_list.append(features)
del training_set_y1
gc.collect()

0

In [11]:
len(y0_feature_list)

5491

In [12]:
y1_feature_list[0].shape

(60, 718)

In [ ]:
# training HMM


startprob = np.zeros(num_state)
startprob[0] = 1

# transition matrix
# transition_matrix = np.array([
#     [0.5, 0.5, 0, 0, 0, 0],
#     [0, 0.5, 0.5, 0, 0, 0],
#     [0, 0, 0.5, 0.5, 0, 0],
#     [0, 0, 0, 0.5, 0.5, 0],
#     [0, 0, 0, 0, 0.5, 0.5],
#     [0, 0, 0, 0, 0, 1.0]
# ])

# Single directional transition matrix
transition_matrix = np.zeros((num_state, num_state))
for i in range(num_state - 1):
    transition_matrix[i, i] = 0.5
    transition_matrix[i, i+1] = 0.5
transition_matrix[-1,-1] =1


list_hmm_models = [] # store each model for one of the two classes
for y_label in range(2): # for class y=0 and y=1
    print('Training for class y =', y_label)
    
    if y_label == 0:
        feature_matrix_list = y0_feature_list
    else:
        feature_matrix_list = y1_feature_list
    
    
    is_the_first_matrix = True
    list_lengths = []
    list_matrix_parts_mfcc_features = [] # for initializing different states by different part of the sequence of frames
    
    count_processed = 0
    for feature_matrix in feature_matrix_list:
        mfcc_matrix = feature_matrix
        mfcc_feature_num, mfcc_length = mfcc_matrix.shape
        list_lengths.append(mfcc_length)

        list_feature_parts_starting_column_nums = np.floor(np.linspace(0, mfcc_length, num_state + 1))
        list_feature_parts_starting_column_nums = list_feature_parts_starting_column_nums.astype(np.int)
        if is_the_first_matrix:
            matrix_all_mfcc_features = mfcc_matrix
            for state_num in range(num_state):
                list_matrix_parts_mfcc_features.append(
                    mfcc_matrix[:, 
                                list_feature_parts_starting_column_nums[state_num] : 
                                list_feature_parts_starting_column_nums[state_num + 1]]
                )

            is_the_first_matrix = False
        else:
            matrix_all_mfcc_features = np.hstack((matrix_all_mfcc_features, mfcc_matrix))
            for state_num in range(num_state):
                part_of_matrix = mfcc_matrix[:, list_feature_parts_starting_column_nums[state_num] : list_feature_parts_starting_column_nums[state_num + 1]]
                list_matrix_parts_mfcc_features[state_num] = np.hstack((list_matrix_parts_mfcc_features[state_num], part_of_matrix))
        count_processed = count_processed + 1
        if count_processed % 100 == 0:
            print('Processed', count_processed, 'out of', len(feature_matrix_list))
    # this time, I initialize the HMM's each state model by the parameters of a GMM
    # each GMM is trained on a different part of the sequence of frames, with the order of the corresponding state
    # e.g., if the number of states is 4, then the feaeture frames of each sample will be divided into 4 parts,
    # then the GMM for the second state is only trained on the data of the second part of each sample
    list_gmm_models = []
    for state_num in range(num_state):
        gmm_model = GaussianMixture(n_components=num_mix, covariance_type='diag', random_state=12345).fit(list_matrix_parts_mfcc_features[state_num].T)
        list_gmm_models.append(gmm_model)
    
    # collect the parameters of all GMMs
    
    # initialize gmm_weights, gmm_means and gmm_covariances with correct shapes
    gmm_weights = np.zeros((num_state, num_mix))
    gmm_means = np.zeros((num_state, num_mix, mfcc_feature_num))
    gmm_covariances = np.zeros((num_state, num_mix, mfcc_feature_num))

    # collect parameters
    for state_num in range(num_state):
        w = list_gmm_models[state_num].weights_
        gmm_weights[state_num] = w.reshape(1, num_mix)
        gmm_means[state_num] = list_gmm_models[state_num].means_
        gmm_covariances[state_num] = list_gmm_models[state_num].covariances_
        
    
#     gmm_model = GaussianMixture(n_components=num_mix, covariance_type='diag', random_state=12345).fit(matrix_all_mfcc_features.T)
#     # reshape GMM's parameters
#     gmm_weights = gmm_model.weights_
#     gmm_weights.reshape(1, num_mix)
#     gmm_weights = np.broadcast_to(gmm_weights, (num_state, num_mix))
    
#     gmm_means = np.expand_dims(gmm_model.means_, 0).repeat(num_state, axis=0)
#     gmm_covariances = np.expand_dims(gmm_model.covariances_, 0).repeat(num_state, axis=0)
    
    hmm_model = GMMHMM(n_components = num_state, n_mix=num_mix, n_iter=60, tol=1e-5, 
                       init_params ='', params ="tmcw",
                       covariance_type='diag', min_covar = 0.0001, random_state=12345, verbose=True)

    hmm_model.startprob_ = startprob
    hmm_model.transmat_ = transition_matrix
    hmm_model.weights_ = gmm_weights
    hmm_model.means_ = gmm_means
    hmm_model.covars_ = gmm_covariances

    hmm_model.fit(matrix_all_mfcc_features.T, np.array(list_lengths))
    
    list_hmm_models.append(hmm_model)



Training for class y = 0
Processed 100 out of 5491
Processed 200 out of 5491
Processed 300 out of 5491
Processed 400 out of 5491
Processed 500 out of 5491
Processed 600 out of 5491
Processed 700 out of 5491
Processed 800 out of 5491
Processed 900 out of 5491
Processed 1000 out of 5491
Processed 1100 out of 5491
Processed 1200 out of 5491
Processed 1300 out of 5491
Processed 1400 out of 5491
Processed 1500 out of 5491
Processed 1600 out of 5491
Processed 1700 out of 5491
Processed 1800 out of 5491
Processed 1900 out of 5491
Processed 2000 out of 5491
Processed 2100 out of 5491
Processed 2200 out of 5491
Processed 2300 out of 5491
Processed 2400 out of 5491
Processed 2500 out of 5491
Processed 2600 out of 5491
Processed 2700 out of 5491
Processed 2800 out of 5491
Processed 2900 out of 5491
Processed 3000 out of 5491
Processed 3100 out of 5491
Processed 3200 out of 5491
Processed 3300 out of 5491
Processed 3400 out of 5491
Processed 3500 out of 5491
Processed 3600 out of 5491
Processed 37

         1  -214799216.9296             +nan
         2  -207029722.4138    +7769494.5158
         3  -204623987.1464    +2405735.2674
         4  -203475337.6124    +1148649.5341
         5  -202826592.6282     +648744.9842
         6  -202495465.6222     +331127.0059
         7  -202276002.8713     +219462.7509
         8  -202064681.4299     +211321.4415
         9  -201866330.8571     +198350.5727
        10  -201674305.7747     +192025.0824
        11  -201373717.7677     +300588.0071
        12  -201090259.5902     +283458.1774
        13  -200891421.4867     +198838.1036
        14  -200773172.1380     +118249.3486
        15  -200699789.9254      +73382.2126
        16  -200641788.1557      +58001.7697
        17  -200600980.5624      +40807.5933
        18  -200550876.9158      +50103.6466
        19  -200495609.4395      +55267.4763
        20  -200454628.6433      +40980.7962
        21  -200425695.7406      +28932.9026
        22  -200400093.2737      +25602.4670
        23

In [ ]:
vis_start = 500
vis_end = 660


In [ ]:

        
#################################################################
# test

count_correct_total = 0
count_test_total = 0
for subject_num in subject_num_list_for_test:
    subject_num_str = str(subject_num + 100000)[-2:]
    y_path = data_set_folder + 'sj_' + subject_num_str + '_Y.mat'
    y_list_mat = h5py.File(y_path, mode='r')
    y_list = np.transpose(y_list_mat['y_list'][:].astype(np.float32))
    
    x_path = data_set_folder + 'sj_' + subject_num_str + '_X.mat'
    x_matrix_mat = h5py.File(x_path, mode='r')
    x_matrix = np.transpose(x_matrix_mat['x_matrix'][:].astype(np.float32))
    num_features, num_samples_of_each_subject = x_matrix.shape
    
    x_matrix_feature_list = []
    for sample_n in range(x_matrix.shape[1]):
        features = np.reshape(x_matrix[:, sample_n], (num_features_after_pca, -1), 'F')
        x_matrix_feature_list.append(features)

    # predict on x_matrix
    prediction_series = []
    for test_sample in x_matrix_feature_list:
            list_score = []
            for model in list_hmm_models:
                score = model.score(test_sample.T)
                list_score.append(score)
            prediction = np.argmax(list_score)
            prediction_series.append(prediction)

    # calculate the accuracy
    count_correct = 0
    for i in range(len(y_list)):
        if y_list[i][0] == prediction_series[i]:
            count_correct = count_correct + 1
    accuracy = count_correct / len(y_list)
    print('The accuracy of testing on subject #' + subject_num_str + ' is ' + str(accuracy))

    count_correct_total = count_correct_total + count_correct
    count_test_total = count_test_total + len(y_list) 


accuracy_total = count_correct_total / count_test_total
print('The total accuracy is', accuracy_total)





# visualize the test results

# repeat some code so that the plot works properly at the first time running this notebook
fig = plt.figure()
plt.rcParams['figure.dpi'] = 300
plt.show()

plt.clf()
fig = plt.figure()
plt.rcParams['figure.dpi'] = 300


# subject 1
plt.subplot(3, 1, 1)

subject_num = subject_num_list_for_test[0]

subject_num_str = str(subject_num + 100000)[-2:]
y_path = data_set_folder + 'sj_' + subject_num_str + '_Y.mat'
y_list_mat = h5py.File(y_path, mode='r')
y_list = np.transpose(y_list_mat['y_list'][:].astype(np.float32))
# obtain the time series of Y
y_time_series = []
for i in range(len(y_list)):
    if y_list[i][0] == 0:
        y_time_series.append(0)
    else:
        y_time_series.append(1)
y_time_series_00 = y_time_series 

y_time_series_array = np.array(y_time_series[vis_start:vis_end])
plt.imshow(y_time_series_array.reshape(1, -1), vmin=0, vmax=1)
plt.xticks(fontsize=5)
plt.yticks(fontsize=5)
# plt.title('Ground truth of subject #' + subject_num_str)
plt.title('Ground truth of testing subject # 1', fontsize=5)


# the time series of my prediction of Y
plt.subplot(3, 1, 2)
prediction_series_array = np.array(prediction_series[vis_start:vis_end])
plt.imshow(prediction_series_array.reshape(1, -1), vmin=0, vmax=1)
plt.xticks(fontsize=5)
plt.yticks(fontsize=5)
plt.title('My prediction of Y', fontsize=5)

# subject 2
plt.subplot(3, 1, 3)

subject_num = subject_num_list_for_test[1]

subject_num_str = str(subject_num + 100000)[-2:]
y_path = data_set_folder + 'sj_' + subject_num_str + '_Y.mat'
y_list_mat = h5py.File(y_path, mode='r')
y_list = np.transpose(y_list_mat['y_list'][:].astype(np.float32))
# obtain the time series of Y
y_time_series = []
for i in range(len(y_list)):
    if y_list[i][0] == 0:
        y_time_series.append(0)
    else:
        y_time_series.append(1)
y_time_series_01 = y_time_series 

y_time_series_array = np.array(y_time_series[vis_start:vis_end])
plt.imshow(y_time_series_array.reshape(1, -1), vmin=0, vmax=1)
plt.xticks(fontsize=5)
plt.yticks(fontsize=5)
# plt.title('Ground truth of subject #' + subject_num_str)
plt.title('Ground truth of testing subject # 2', fontsize=5)